# Applying Classification Modeling
The goal of this week's assessment is to find the model which best predicts whether or not a person will default on their bank loan. In doing so, we want to utilize all of the different tools we have learned over the course: data cleaning, EDA, feature engineering/transformation, feature selection, hyperparameter tuning, and model evaluation. 


#### Data Set Information:

This research aimed at the case of customers default payments in Taiwan and compares the predictive accuracy of probability of default among six data mining methods. From the perspective of risk management, the result of predictive accuracy of the estimated probability of default will be more valuable than the binary result of classification - credible or not credible clients. Because the real probability of default is unknown, this study presented the novel Sorting Smoothing Method to estimate the real probability of default. With the real probability of default as the response variable (Y), and the predictive probability of default as the independent variable (X), the simple linear regression result (Y = A + BX) shows that the forecasting model produced by artificial neural network has the highest coefficient of determination; its regression intercept (A) is close to zero, and regression coefficient (B) to one. Therefore, among the six data mining techniques, artificial neural network is the only one that can accurately estimate the real probability of default. 

- NT is the abbreviation for New Taiwain. 


#### Attribute Information:

This research employed a binary variable, default payment (Yes = 1, No = 0), as the response variable. This study reviewed the literature and used the following 23 variables as explanatory variables: 
- X1: Amount of the given credit (NT dollar): it includes both the individual consumer credit and his/her family (supplementary) credit. 
- X2: Gender (1 = male; 2 = female). 
- X3: Education (1 = graduate school; 2 = university; 3 = high school; 4 = others). 
- X4: Marital status (1 = married; 2 = single; 3 = others). 
- X5: Age (year). 
- X6 - X11: History of past payment. We tracked the past monthly payment records (from April to September, 2005) as follows: 
    - X6 = the repayment status in September, 2005; 
    - X7 = the repayment status in August, 2005; . . .;
    - etc...
    - X11 = the repayment status in April, 2005. 
    - The measurement scale for the repayment status is: -1 = pay duly; 1 = payment delay for one month; 2 = payment delay for two months; . . .; 8 = payment delay for eight months; 9 = payment delay for nine months and above. 
- X12-X17: Amount of bill statement (NT dollar). 
    - X12 = amount of bill statement in September, 2005;
    - etc...
    - X13 = amount of bill statement in August, 2005; . . .; 
    - X17 = amount of bill statement in April, 2005. 
- X18-X23: Amount of previous payment (NT dollar). 
    - X18 = amount paid in September, 2005; 
    - X19 = amount paid in August, 2005; . . .;
    - etc...
    - X23 = amount paid in April, 2005. 




You will fit three different models (KNN, Logistic Regression, and Decision Tree Classifier) to predict credit card defaults and use gridsearch to find the best hyperparameters for those models. Then you will compare the performance of those three models on a test set to find the best one.  


## Process/Expectations

- You will be working in pairs for this assessment

### Please have ONE notebook and be prepared to explain how you worked in your pair.

1. Clean up your data set so that you can perform an EDA. 
    - This includes handling null values, categorical variables, removing unimportant columns, and removing outliers.
2. Perform EDA to identify opportunities to create new features.
    - [Great Example of EDA for classification](https://www.kaggle.com/stephaniestallworth/titanic-eda-classification-end-to-end) 
    - [Using Pairplots with Classification](https://towardsdatascience.com/visualizing-data-with-pair-plots-in-python-f228cf529166)
3. Engineer new features. 
    - Create polynomial and/or interaction features. 
    - Additionaly, you must also create **at least 2 new features** that are not interactions or polynomial transformations. 
        - *For example, you can create a new dummy variable that based on the value of a continuous variable (billamount6 >2000) or take the average of some past amounts.*
4. Perform some feature selection. 
    
5. You must fit **three** models to your data and tune **at least 1 hyperparameter** per model. 
6. Using the F-1 Score, evaluate how well your models perform and identify your best model.
7. Using information from your EDA process and your model(s) output provide insight as to which borrowers are more likely to deafult


In [76]:
# import libraries

import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 50)

## 1. Data Cleaning

In [2]:
df = pd.read_csv('training_data.csv' , index_col=0)

In [3]:
df.Y.value_counts()

0                             17471
1                              5028
default payment next month        1
Name: Y, dtype: int64

In [4]:
df.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X15,X16,X17,X18,X19,X20,X21,X22,X23,Y
28835,220000,2,1,2,36,0,0,0,0,0,...,221193,181859,184605,10000,8018,10121,6006,10987,143779,1
25329,200000,2,3,2,29,-1,-1,-1,-1,-1,...,326,326,326,326,326,326,326,326,326,0
18894,180000,2,1,2,27,-2,-2,-2,-2,-2,...,0,0,0,0,0,0,0,0,0,0
690,80000,1,2,2,32,0,0,0,0,0,...,43882,42256,42527,1853,1700,1522,1548,1488,1500,0
6239,10000,1,2,2,27,0,0,0,0,0,...,5444,2639,2697,2000,1100,600,300,300,1000,1


In [5]:
df['X6']

28835     0
25329    -1
18894    -2
690       0
6239      0
         ..
16247     0
2693     -1
8076      1
20213    -1
7624      1
Name: X6, Length: 22500, dtype: object

In [6]:
df.dtypes

X1     object
X2     object
X3     object
X4     object
X5     object
X6     object
X7     object
X8     object
X9     object
X10    object
X11    object
X12    object
X13    object
X14    object
X15    object
X16    object
X17    object
X18    object
X19    object
X20    object
X21    object
X22    object
X23    object
Y      object
dtype: object

In [7]:
# Split data to be used in the models
# Create matrix of features
X = df.drop('Y', axis = 1) # grabs everything else but 'Survived' is this suspoesed to be survived?


# Create target variable
y = df['Y'] # y is the column we're trying to predict

In [8]:
df['X3'].value_counts()

2            10516
1             7919
3             3713
5              208
4               90
6               42
0               11
EDUCATION        1
Name: X3, dtype: int64

## 2. EDA

In [9]:
df.drop('ID', axis = 0, inplace = True)

In [10]:
df = df.apply(lambda x: x.astype(float))

In [11]:
# Split data to be used in the models
# Create matrix of features
X = df.drop('Y', axis = 1) # grabs everything else but 'Survived'

In [12]:
# Create target variable
y = df['Y'] # y is the column we're trying to predict

In [13]:
## For our Marital Status column, we have 3 groups in our key but 4 in the dataframe
## We need to figure out where 0 belongs

## ! is married, 2 is single, 3 is others

df.groupby('X4')['X5'].mean()

X4
0.0    37.590909
1.0    40.027857
2.0    31.412606
3.0    42.893162
Name: X5, dtype: float64

In [14]:
## 0 appears to belong to 1 aka the married group but I should investigate further

In [15]:
df.groupby('X4')['X1'].mean()

## We have 4 groups in our key for Education Level but 7 in our dataframe

X4
0.0    140681.818182
1.0    181372.437469
2.0    156242.115417
3.0    103888.888889
Name: X1, dtype: float64

In [16]:
df.groupby('X3')['X5'].mean()

##6 and 0 look like they belong to 3 aka high school education
## 5 might belong to 2 aka college education but I should investigate further

X3
0.0    40.545455
1.0    34.231847
2.0    34.658806
3.0    40.157285
4.0    34.666667
5.0    35.937500
6.0    43.904762
Name: X5, dtype: float64

In [17]:
df.groupby('X3')['X1'].mean()

## 1 graduate, 2 college, 3 high school, 4 others

X3
0.0    220000.000000
1.0    213389.316833
2.0    146419.360974
3.0    125641.712901
4.0    230000.000000
5.0    161951.923077
6.0    135000.000000
Name: X1, dtype: float64

In [18]:
df['X3']

28835    1.0
25329    3.0
18894    1.0
690      2.0
6239     2.0
        ... 
16247    2.0
2693     1.0
8076     3.0
20213    3.0
7624     1.0
Name: X3, Length: 22499, dtype: float64

In [19]:
df[(df['X3'] == 5) | (df['X3'] == 6) | (df['X3'] == 0)]['X4'].value_counts()

1.0    139
2.0    117
3.0      5
Name: X4, dtype: int64

In [20]:
df[df['X4'] == 0]['X5'].mean()

37.59090909090909

In [21]:
conditions = [
    df['X3'].eq(0),
    df['X3'].eq(1),
    df['X3'].eq(2),
    df['X3'].eq(3),
    df['X3'].eq(4),
    df['X3'].eq(5),
    df['X3'].eq(6),
]

choices = [
    1,
    0,
    0,
    0,
    0,
    1,
    1,   
]

df['X3_undefined'] = np.select(conditions,choices)

In [22]:
df['X4_undefined'] = np.where(df['X4'] == 0, 1,0)

In [23]:
df['X3_undefined'].value_counts()

0    22238
1      261
Name: X3_undefined, dtype: int64

In [24]:
df['X4_undefined'].value_counts()

0    22455
1       44
Name: X4_undefined, dtype: int64

In [25]:
df['X4'].value_counts()

2.0    12026
1.0    10195
3.0      234
0.0       44
Name: X4, dtype: int64

In [26]:
df['X3'].value_counts()

2.0    10516
1.0     7919
3.0     3713
5.0      208
4.0       90
6.0       42
0.0       11
Name: X3, dtype: int64

## 3. Feature Engineering

### Remaining balances after each monthly payment

In [27]:
df['bal_september'] = df['X12'] - df['X18'] #amount left to pay September
df['bal_august'] = df['X13'] - df['X19'] #amount left to pay August
df['bal_july'] = df['X14'] - df['X20'] #amount left to pay July
df['bal_june'] = df['X15'] - df['X21'] #amount left to pay June
df['bal_may'] = df['X16'] - df['X22'] #amount left to pay July
df['bal_april'] = df['X17'] - df['X23'] #amount left to pay April

### Monthly credit utilization

In [28]:
df['util_september'] = df['bal_september'] / df['X1'] #September credit utilization
df['util_august'] = df['bal_august'] / df['X1'] #August credit utilization
df['util_july'] = df['bal_july'] / df['X1'] #July credit utilization
df['util_june'] = df['bal_june'] / df['X1'] #June credit utilization
df['util_may'] = df['bal_may'] / df['X1'] #July credit utilization
df['util_april'] = df['bal_april'] / df['X1'] #April credit utilization

### Utilization Trend

In [29]:
# # test = [0.12,0.23,0.34,0.45,0.6]
# test = [0.6, 0.5, 0.54, 0.2, 0.3]

In [83]:
# for index, i in enumerate(test):
#     if index == 0:
#         trend = 0
#     else:
#         trend += i - test[index - 1]
# trend

-0.3

In [30]:
def util_trend(df,cols):
    for index, i in enumerate(cols):
        if index == 0:
            trend = 0
        else:
            trend += df[i] - df[cols[index - 1]]
    return trend

In [32]:
prefix = ['util']
months = ['september', 'august', 'july', 'june', 'may', 'april']
i = [prefix[0] + "_" + x for x in months]
df['util_trend'] = util_trend(df, i) #the overall trend of the credit utlization

In [33]:
i

['util_september',
 'util_august',
 'util_july',
 'util_june',
 'util_may',
 'util_april']

### Average Utilization

In [34]:
df['avg_util'] = (df[i[0]] + df[i[1]] + df[i[2]] + df[i[3]] + df[i[4]] + df[i[5]]) / 6 #average credit utilization

### Payment History

In [35]:
df['payment_hist'] = df.apply(lambda x:[x['X6'], x['X7'], x['X8'], x['X9'], x['X10'], x['X11']], axis = 1) 

looks like -2 is a value but i don't know what that means. Maybe they prepaid? Also, you can't be two months behind when you were up to date the month before.

According to Kaggle this is a possible interpretation:

-2 = Balance paid in full and no transactions this period (we may refer to this credit card account as having been 'inactive' this period)

-1 = Balance paid in full, but account has a positive balance at end of period due to recent transactions for which payment has not yet come due

0 = Customer paid the minimum due amount, but not the entire balance. I.e., the customer paid enough for their account to remain in good standing, but did revolve a balance

### Account Status

In [36]:
df['account_status'] = df['payment_hist'].apply(lambda lst:1 if max(set(lst), key=lst.count) > 0 else 0)

In [68]:
# 1 = bad standing, payments are not up to date for a majority of the time 0 = good standing the majority of time

### Ever owed more than thier credit limit

In [67]:
df['excess_debt'] = 1 * df.apply(lambda x: x['X12':'X17'] > x['X1'], axis = 1).any(axis = 1)

In [68]:
df['excess_debt'].value_counts()

0    19600
1     2899
Name: excess_debt, dtype: int64

### Continually owed more than thier credit limit

In [69]:
df['consistent_excess_debt'] = 1 * (df.apply(lambda x: x['X12':'X17'] > x['X1'], axis = 1).sum(axis = 1) > 1)

In [71]:
df['consistent_excess_debt'].value_counts()

0    20931
1     1568
Name: consistent_excess_debt, dtype: int64

In [77]:
df.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,Y,X3_undefined,X4_undefined,bal_september,bal_august,bal_july,bal_june,bal_may,bal_april,util_september,util_august,util_july,util_june,util_may,util_april,util_trend,avg_util,payment_hist,account_status,excess_debt,consistent_excess_debt
28835,220000.0,2.0,1.0,2.0,36.0,0.0,0.0,0.0,0.0,0.0,0.0,222598.0,222168.0,217900.0,221193.0,181859.0,184605.0,10000.0,8018.0,10121.0,6006.0,10987.0,143779.0,1.0,0,0,212598.0,214150.0,207779.0,215187.0,170872.0,40826.0,0.966355,0.973409,0.944450,0.978123,0.776691,0.185573,-0.780782,0.804100,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",0,1,1
25329,200000.0,2.0,3.0,2.0,29.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,326.0,326.0,326.0,326.0,326.0,326.0,326.0,326.0,326.0,326.0,326.0,326.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0]",0,0,0
18894,180000.0,2.0,1.0,2.0,27.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,"[-2.0, -2.0, -2.0, -2.0, -2.0, -2.0]",0,0,0
690,80000.0,1.0,2.0,2.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,51372.0,51872.0,47593.0,43882.0,42256.0,42527.0,1853.0,1700.0,1522.0,1548.0,1488.0,1500.0,0.0,0,0,49519.0,50172.0,46071.0,42334.0,40768.0,41027.0,0.618988,0.627150,0.575887,0.529175,0.509600,0.512837,-0.106150,0.562273,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",0,0,0
6239,10000.0,1.0,2.0,2.0,27.0,0.0,0.0,0.0,0.0,0.0,0.0,8257.0,7995.0,4878.0,5444.0,2639.0,2697.0,2000.0,1100.0,600.0,300.0,300.0,1000.0,1.0,0,0,6257.0,6895.0,4278.0,5144.0,2339.0,1697.0,0.625700,0.689500,0.427800,0.514400,0.233900,0.169700,-0.456000,0.443500,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",0,0,0


## 4. Feature Selection

## 5. Model Fitting and Hyperparameter Tuning
KNN, Logistic Regression, Decision Tree

## 6. Model Evaluation

## 7. Final Model